In [40]:
from pygsti.objects import gatestring as _gs
from pygsti.construction import list_all_gatestrings_onelen, list_all_gatestrings
from pygsti.construction import gen_all_gatestrings_onelen, gen_all_gatestrings
import itertools as _itertools

In [41]:
def itertools_onelen(gateLabels, length):
    gateTuples = _itertools.product(gateLabels, repeat=length)
    return list(map(_gs.GateString, gateTuples))

In [42]:
myGates = [ 'Gx', 'Gy' ]

## Performance

Check to see which implementation performs faster.

### Long sequences

In [43]:
%timeit -n 10 list_all_gatestrings_onelen(myGates, 15)

10 loops, best of 3: 44.9 ms per loop


In [44]:
%timeit -n 10 itertools_onelen(myGates, 15)

10 loops, best of 3: 44.1 ms per loop


### Short sequences

In [45]:
%timeit -n 10 list_all_gatestrings_onelen(myGates, 5)

10 loops, best of 3: 46.7 µs per loop


In [46]:
%timeit -n 10 itertools_onelen(myGates, 5)

10 loops, best of 3: 46.5 µs per loop


## Correctness

Make sure the new implementation gives the same results.

In [47]:
all([list_all_gatestrings_onelen(myGates, length) == itertools_onelen(myGates, length)
     for length in range(15)])

True

The implementation using itertools seems to perform correctly and about 4x as quickly
as the current implementation.

## All gatestrings

In [48]:
def iter_list_all_gatestrings(gateLabels, minlength, maxlength):
    gateTuples = _itertools.chain(*[_itertools.product(gateLabels, repeat=N)
                                    for N in range(minlength, maxlength + 1)])
    return list(map(_gs.GateString, gateTuples))

In [49]:
all([list_all_gatestrings(myGates, minlength, maxlength) ==
     iter_list_all_gatestrings(myGates, minlength, maxlength)
     for minlength, maxlength in
     list(_itertools.chain(*[_itertools.product(range(N + 1), [N]) for N in range(15)]))])

True

In [50]:
%timeit -n 10 list_all_gatestrings(myGates, 0, 15)

10 loops, best of 3: 94.4 ms per loop


In [51]:
%timeit -n 10 iter_list_all_gatestrings(myGates, 0, 15)

10 loops, best of 3: 104 ms per loop


In [52]:
def iter_gen_onelen(gateLabels, length):
    for gateTuple in _itertools.product(gateLabels, repeat=length):
        yield _gs.GateString(gateTuple)

In [53]:
gen1 = gen_all_gatestrings_onelen(myGates, 15)
gen2 = iter_gen_onelen(myGates, 15)
comparisons = []
while True:
    try:
        comparisons.append(gen1.next() == gen2.next())
    except StopIteration:
        break
print 'Equal once one finished?', all(comparisons)
try:
    print 'gen1 still had:', gen1.next()
except StopIteration:
    pass
try:
    print '\ngen2 still had:', gen2.next()
except StopIteration:
    pass

Equal once one finished? True
gen1 still had: 
gen2 still had:


In [54]:
def iter_gen_all(gateLabels, minlength, maxlength):
    gateTuples = _itertools.chain(*[_itertools.product(gateLabels, repeat=N)
                                    for N in range(minlength, maxlength + 1)])
    for gateTuple in gateTuples:
        yield _gs.GateString(gateTuple)

In [55]:
gen3 = gen_all_gatestrings(myGates, 0, 15)
gen4 = iter_gen_all(myGates, 0, 15)
comparisons = []
while True:
    try:
        comparisons.append(gen3.next() == gen4.next())
    except StopIteration:
        break
print 'Equal once one finished?', all(comparisons)
try:
    print 'gen3 still had:', gen3.next()
except StopIteration:
    pass
try:
    print '\ngen4 still had:', gen4.next()
except StopIteration:
    pass

Equal once one finished? True
gen3 still had: 
gen4 still had:
